In [7]:
import requests 
import json 
import pandas as pd

# Turning this into a full function now 
def GetConstructorStanding(years, rounds):
  url = "http://ergast.com/api/f1/{year}/{round}/constructorStandings.json".format(year = str(years), round = str(rounds))
  response = requests.get(url)
  stats = json.loads(response.text)
  column_list = ["Year", "Round", "Constructor", "Constructor_Points", "Constructor_Wins"]
  # make a new dataframe to write to 
  export = pd.DataFrame(columns=column_list)

  i = 0
  length = int(stats['MRData']['total'])

  while i < length: 
    # creating an if statement to stop the loop if
    # nothing is there 
    try:
      # creating an empty dict
      conStat = {}

      # now assigning what I need to this dict
      conStat['Year'] = stats['MRData']['StandingsTable']['season']
      conStat['Round'] = stats['MRData']['StandingsTable']['round']
      conStat['Constructor'] = stats['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings'][i]['Constructor']['name']
      conStat['Constructor_Points'] = stats['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings'][i]['points']
      conStat['Constructor_Wins'] = stats['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings'][i]['wins']

      # now creating a dataframe to work with this. 
      quick_df = pd.DataFrame(conStat, index= [0])
      export = pd.concat([export, quick_df], ignore_index=True)
      i+=1
    except IndexError:
      return export
  return export 


In [8]:
GetConstructorStanding(2020, 5)

,Year,Round,Constructor,Constructor_Points,Constructor_Wins
0,2020,5,Mercedes,180,4
1,2020,5,Red Bull,113,1
2,2020,5,Ferrari,55,0
3,2020,5,McLaren,53,0
4,2020,5,Racing Point,41,0
5,2020,5,Renault,36,0
6,2020,5,AlphaTauri,14,0
7,2020,5,Alfa Romeo,2,0
8,2020,5,Haas F1 Team,1,0
9,2020,5,Williams,0,0


In [10]:
def cleandata(df):
  '''
    The goal is to take in the messy dataframe and clean it up by creating data
    frames that contain only the data and returns a cleaned up dataframe. 

    TODO: Find a way to get this function working with a for loop. 
  '''
  df1 = pd.DataFrame()
  df2 = pd.DataFrame()
  df3 = pd.DataFrame()
  df4 = pd.DataFrame()
  df5 = pd.DataFrame()

  df1["Number"] = df['PermanentNumber']
  df2['First_Name'] = df['First_Name']
  df3['Last_Name'] = df['Last_Name']
  df4["DOB"] = df['DOB']
  df5['Nationality'] = df['Nationality']

  df1 = df1.dropna()
  df2 = df2.dropna().reset_index()
  df3 = df3.dropna().reset_index()
  df4 = df4.dropna().reset_index()
  df5 = df5.dropna().reset_index()

  new_df= pd.DataFrame()
  new_df["Number"] = df1['Number']
  new_df['First_Name'] = df2['First_Name']
  new_df['Last_Name'] = df3['Last_Name']
  new_df["DOB"] = df4['DOB']
  new_df['Nationality'] = df5['Nationality']
  return new_df

In [11]:
# Testing to see how to implement being one lap behind pace. 
url = "http://ergast.com/api/f1/2020/2/results.json"
response = requests.get(url)
laps = json.loads(response.text)
# This query is essentially grabbing the driver information for whoever ended with a +1 lap
# I need to make it a true/false statement. 
# EVEN BETTER I GOT WHERE I CAN GET THEIR FASTEST LAP INFORMATION.... 

laps['MRData']['RaceTable']['Races'][0]['Results'][1]['Time']['time']

'+13.719'

In [29]:
# Creating a mock dataframe that will contain the information I would like to keep. 

finish_mock = {}
finish_mock1 = {}
finish_mock1['First_Name'] = laps['MRData']['RaceTable']['Races'][0]['Results'][0]['Driver']['givenName']
finish_mock1['Last_Name'] = laps['MRData']['RaceTable']['Races'][0]['Results'][0]['Driver']['familyName']
finish_mock['Driver'] = finish_mock1['First_Name'] + " " + finish_mock1['Last_Name']
finish_mock['FastestLap'] = laps['MRData']['RaceTable']['Races'][0]['Results'][0]['FastestLap']['lap']
finish_mock['FastestTime'] = laps['MRData']['RaceTable']['Races'][0]['Results'][0]['FastestLap']['Time']['time']
finish_mock['FastestLapAvgSpeedKPH'] = laps['MRData']['RaceTable']['Races'][0]['Results'][0]['FastestLap']['AverageSpeed']['speed']
finish_mock['status'] = laps['MRData']['RaceTable']['Races'][0]['Results'][0]['status']

finish_mock_df = pd.DataFrame(finish_mock, index= [0])
finish_mock_df

,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status
0,Lewis Hamilton,68,1:06.719,232.989,Finished


In [28]:
# Now I want to combine the first and last name columns. 

finish_mock_df['Driver'] = finish_mock_df[["First_Name", "Last_Name"]].apply(lambda x: " ".join(x), axis=1)
finish_mock_df = finish_mock_df.drop(["First_Name", "Last_Name"], axis=1)
finish_mock_df

,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status
0,Lewis Hamilton,68,1:06.719,232.989,Finished


In [32]:
# I got the bare bones working, time to put this into a loop that will grab that information for all the drivers of that race 

def getLapInfo(year, round):
    """ 
        This function takes in the year and round of a f1 race and returns the race information beyond what place they have               finished. It returns info such as if they finished a lap or more behind, or out of the race for any reason, their fastest         lap and the time the driver is behind the race leader. 
    """
    url = "http://ergast.com/api/f1/{years}/{rounds}/results.json".format(years=year, rounds=round)
    response = requests.get(url)
    laps = json.loads(response.text)   
    column = ['Year', 'Round', 'Driver','FastestLap','FastestTime','FastestLapAvgSpeedKPH']
    # Grabbing the amount of drivers 
    total_drivers = int(laps['MRData']['total'])
    export = pd.DataFrame(columns=column)

    # Setting i variable for try/while loop
    i = 0 

    # Now using a try/except/while loop to get what information I need 
    try:
        while i < total_drivers:
            finish_mock = {}
            finish_mock1 = {}
            finish_mock['Year'] = year
            finish_mock['Round'] = round
            finish_mock1['First_Name'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['Driver']['givenName']
            finish_mock1['Last_Name'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['Driver']['familyName']
            finish_mock['Driver'] = finish_mock1["First_Name"] + " " + finish_mock1["Last_Name"]
            finish_mock['status'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['status']

            try: 
                finish_mock['FastestLap'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['FastestLap']['lap']
                finish_mock['FastestTime'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['FastestLap']['Time']['time']
                finish_mock['FastestLapAvgSpeedKPH'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['FastestLap']['AverageSpeed']['speed']
                finish_mock['TimeBehind'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['Time']['time']
                finish_mock_df = pd.DataFrame(finish_mock, index=[0])
                export = pd.concat([export, finish_mock_df], ignore_index=True)

            except KeyError:
                finish_mock['TimeBehind'] = laps['MRData']['RaceTable']['Races'][0]['Results'][i]['status']
                finish_mock['FastestLap'] = None
                finish_mock['FastestTime'] = None
                finish_mock['FastestLapAvgSpeedKPH'] = None
                finish_mock_df = pd.DataFrame(finish_mock, index=[0])
                export = pd.concat([export, finish_mock_df], ignore_index=True)
            i += 1 
    except IndexError:
        return export 

    return export 

In [33]:
# getting the final dataframe to see how it would look
df = getLapInfo(2020, 6)
df
# TODO: Get this function to merge with the main dataframe! 

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind
0,2020,6,Lewis Hamilton,63,1:19.822,209.942,Finished,1:31:45.279
1,2020,6,Max Verstappen,53,1:21.477,205.677,Finished,+24.177
2,2020,6,Valtteri Bottas,66,1:18.183,214.343,Finished,+44.752
3,2020,6,Lance Stroll,None,None,None,+1 Lap,+1 Lap
4,2020,6,Sergio Pérez,None,None,None,+1 Lap,+1 Lap
5,2020,6,Carlos Sainz,None,None,None,+1 Lap,+1 Lap
6,2020,6,Sebastian Vettel,None,None,None,+1 Lap,+1 Lap
7,2020,6,Alexander Albon,None,None,None,+1 Lap,+1 Lap
8,2020,6,Pierre Gasly,None,None,None,+1 Lap,+1 Lap
9,2020,6,Lando Norris,None,None,None,+1 Lap,+1 Lap


# Change of pace
I'm going to take a complete left turn. 

I think it would be better to have have to data I get be a JSON file instead of an xml. It is faster to parse AND I believe it would be easier to work with. 

In [36]:
# Quick test Seeing how I would get a JSON 
import json

j = 2016
length = 0

url = "http://ergast.com/api/f1/{year}/{round}/driverStandings.json"
url1 = url.format(year = str(j), round = str(length))

url1

'http://ergast.com/api/f1/2016/0/driverStandings.json'

In [37]:
import requests 

response = requests.get(url1)
standing = json.loads(response.text)

In [34]:
import pandas as pd 

df = pd.DataFrame()

column_list = ['Driver', 'Nationality', 'Number', 
               'Points', 'Position', 'Wins']

# New Data Extraction Function

Using what I've learned from my spotify project, I'm going to create a function that will take my scrapped data and return it as a pandas dataframe. 

In [24]:
# live test to see how I'm going to parse the data to get what I need for each column
years = 2020
rounds = 3
url = "http://ergast.com/api/f1/{year}/{round}/driverStandings.json"
url1 = url.format(year = str(years), round = str(rounds))
standing['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'][1]['Driver']['nationality']

'British'

In [41]:
# This is a test function to get test data. 
# Keeping it simple and controlled
def GetStandings(years, rounds):
  '''
    Simple function that returns a dataframe that contains the 
    year and round of a race and how the drivers place on it. 
  '''
  # making sure the inputs are strings 
  url = "http://ergast.com/api/f1/{year}/{round}/driverStandings.json"
  url1 = url.format(year = str(years), round = str(rounds))

  # now creating my dataframe 
  df = pd.DataFrame()

  # getting my dict that I will turn into a dataframe
  response = requests.get(url1)
  standing = json.loads(response.text)

  # with that info creating my columns 
  column_list = ['Year', 'Round', 'Driver', 
                 'Track', 'Points', 'Position', 
                 'Wins']

  # now with that done time to take my dict and make 
  # it a data frame 
  i = 0

  # this determines how long the loop goes so we don't get an index error!
  length = int(standing['MRData']['total'])
  
  # This is going to be another variable, which will 
  # Get the track name of the round that it's in 
  url = "http://ergast.com/api/f1/{year}/{round}/qualifying.json".format(year = str(years), round = str(rounds))
  response = requests.get(url)
  track = json.loads(response.text)
  # This will be my final dataframe
  standing_df = pd.DataFrame(columns=column_list)

  # Using a while loop since every season has a different number of drivers
  # Want this program to work in the long run. 
  while i < length:
    driver_features = {}
    driver_name = {}
    # getting the data I need
    driver_features['Year'] = int(years)
    driver_features['Round'] = int(rounds)
    driver_name['First_Name'] = standing['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'][i]['Driver']['givenName']
    driver_name['Last_Name'] = standing['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'][i]['Driver']['familyName']
    driver_features['Driver'] = driver_name['First_Name'] + ' ' + driver_name['Last_Name']
    driver_features['Track'] = track["MRData"]["RaceTable"]["Races"][0]['Circuit']['circuitId']
    driver_features['Points'] = standing['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'][i]['points']
    driver_features['Position'] = standing['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'][i]['position']
    driver_features['Wins'] = standing['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'][i]['wins']
    i+=1

    standing_df1 = pd.DataFrame(driver_features, index= [0])
    standing_df = pd.concat([standing_df, standing_df1], ignore_index=True)
  return standing_df

In [42]:
import requests
import json 
import pandas as pd 

url = "http://ergast.com/api/f1/{year}/{round}/qualifying.json".format(year = "2016", round = '3')

response = requests.get(url)
test = json.loads(response.text)
test['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId']

'shanghai'

My current problem is that no matter how I set the loop up, I seem to only get one result with my dataframe. 

fixed it by creating a dataframe outside the loop and concatted the newly created one to the other one until the loop is done. it was overwriting itself the entire time. LOL

my second problem is that sometimes the total number of drivers is less than the actual number

The fix was simple, an overthinking part on my side. forgot that the json I'm working with already had the number of drivers in it already. used that to count the total instead. 


In [43]:
GetStandings(2020, 3)

,Year,Round,Driver,Track,Points,Position,Wins
0,2020,3,Lewis Hamilton,hungaroring,63,1,2
1,2020,3,Valtteri Bottas,hungaroring,58,2,1
2,2020,3,Max Verstappen,hungaroring,33,3,0
3,2020,3,Lando Norris,hungaroring,26,4,0
4,2020,3,Alexander Albon,hungaroring,22,5,0
5,2020,3,Sergio Pérez,hungaroring,22,6,0
6,2020,3,Charles Leclerc,hungaroring,18,7,0
7,2020,3,Lance Stroll,hungaroring,18,8,0
8,2020,3,Carlos Sainz,hungaroring,15,9,0
9,2020,3,Sebastian Vettel,hungaroring,9,10,0


Now since I got the simple function done, time to make the big one again. This time it's much faster and much smaller. 

I want this function to return the results of the last 5 years 

In [51]:
from datetime import datetime
def GetLast5Years():
  '''
  This function takes the driver standings from the last 
  5 years and returns an exported dataframe to work with. 
  '''
  # bringing in the column list once again 
  # TODO, in VSCODE make sure I can easily import it 
  # to save memory when running the full app. 
  column_list = ['Year', 'Round', 'Driver', 
                 'Track', 'Points', 'Position', 
                 'Wins']
  column_list2 = ["Year", "Round", "Driver", 
                  "Q1_Time", "Q2_Time", "Q3_Time"]
  column_list3 = ['Year', 'Round', 'Driver','FastestLap','FastestTime','FastestLapAvgSpeedKPH']
  column_list4 = ["Year", "Round", "Constructor", "Constructor_Points", "Constructor_Wins"]
  # making a new dataframe that will contain all of my variables. 
  export = pd.DataFrame(columns=column_list)
  export1 = pd.DataFrame(columns= column_list2)
  export_Con = pd.DataFrame(columns= column_list4) # Used to be export 2
  export_Lap = pd.DataFrame(columns= column_list3) # Used to be export 3

  # Starting with my first for loop, witch will loop through 
  # the last 5 years from the current date 
  today = datetime.today().year
  # A temporary fix since the 2021 season hasn't started yet
  # TODO, find a way to have to function work if 
  # The season hasn't started yet. 
  # Probs just have it do a precheck, if it returns an error
  # Skip the year and continue. 
  if today == 2021:
    today -= 1
  year_list = []
  # Creating my boundries for the loop 
  for i in range(5):
    year_list.append(today)
    today -= 1

  for race in year_list:
    # Grabbing the length of the inner loop. 
    url4 = "http://ergast.com/api/f1/{year}.json".format(year=str(race))
    response = requests.get(url4)
    test = json.loads(response.text)
    length = int(test['MRData']['total'])
    

    for k in range(length):
      # The goal of this loop is to grab the standings for each round 
      # and add it to the dataframe. 
      temp4 = getLapInfo(race, k)
      temp2 = get_qualifying(race, k)
      temp = GetStandings(race, k)
      temp3 = GetConstructorStanding(race, k)
      # Merging the Constructor dataframes
      export_Con = pd.concat([export_Con, temp3], ignore_index=True)
      # Merging the additional driver info 
      export_Lap = pd.concat([export_Lap, temp4], ignore_index=True)
      # Merging the base driver info. 
      export = pd.concat([export, temp], ignore_index=True)
      export1 = pd.concat([export1, temp2], ignore_index=True)
      # Final combination of the dataframes. 
  
  done = pd.merge(export1, export, on=['Year', 'Round', 'Driver'])
  final_export = pd.merge(export_Lap, done, on=['Year', 'Round', 'Driver'])    
  return final_export, export_Con

# Getting the qualifying times 

This function will grab all of the Q1 times for each driver

TODO: If I do have any spare time, have this function take the drivers fastest time instead of just their Q1 time. Problem is that it will make the function just a bit slower....


In [52]:
# Writing a function that will get the qualifying times for Q1
def get_qualifying(years, rounds):
  # making my columns list
  column_list = ["Year", "Round", "Driver", "Q1_Time", "Q2_Time", "Q3_Time", "Constructor"]
  # make a new dataframe to write to 
  export = pd.DataFrame(columns=column_list)
  
  # now getting the dict
  url = "http://ergast.com/api/f1/{year}/{round}/qualifying.json".format(year = str(years), round = str(rounds))
  response = requests.get(url)
  qual = json.loads(response.text)

  # Setting the varible for our while loop
  i = 0 

  length = int(qual['MRData']['total'])

  # making a while loop 
  while i < length: 
    # creating an if statement to stop the loop if
    # nothing is there 
    try:
      # creating an empty dict
      qual_feat = {}
      qual_name = {}

      # now assigning what I need to this dict
      qual_feat["Year"] = int(years)
      qual_feat["Round"] = (int(rounds) + 1)
      qual_name["First_Name"] = qual["MRData"]["RaceTable"]["Races"][0]["QualifyingResults"][i]['Driver']['givenName']
      qual_name["Last_Name"] = qual['MRData']['RaceTable']['Races'][0]['QualifyingResults'][i]['Driver']['familyName']
      qual_feat["Driver"] = qual_name["First_Name"] + " " + qual_name['Last_Name']
      qual_feat["Q1_Time"] = qual['MRData']['RaceTable']['Races'][0]['QualifyingResults'][i]['Q1']
      try:
        qual_feat['Q2_Time'] = qual['MRData']['RaceTable']['Races'][0]['QualifyingResults'][i]['Q2']
      except KeyError:
        pass
      try:
        qual_feat["Q3_Time"] = qual['MRData']['RaceTable']['Races'][0]['QualifyingResults'][i]['Q3']
      except KeyError:
        pass 
      qual_feat["Constructor"] = qual['MRData']['RaceTable']['Races'][0]['QualifyingResults'][i]['Constructor']['name']

      # now creating a dataframe to work with this. 
      quick_df = pd.DataFrame(qual_feat, index= [0])
      export = pd.concat([export, quick_df], ignore_index=True)
      i+=1
    except IndexError:
      return export
  return export 

In [53]:
df, test = GetLast5Years()
df

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Position,Wins
0,2020,1,Valtteri Bottas,68,1:07.657,229.758,Finished,1:30:55.739,1:04.111,1:03.015,1:02.939,Mercedes,red_bull_ring,25,1,1
1,2020,1,Charles Leclerc,64,1:07.901,228.933,Finished,+2.700,1:04.500,1:04.041,1:03.923,Ferrari,red_bull_ring,18,2,0
2,2020,1,Lando Norris,71,1:07.475,230.378,Finished,+5.491,1:04.606,1:03.819,1:03.626,McLaren,red_bull_ring,16,3,0
3,2020,1,Lewis Hamilton,67,1:07.712,229.572,Finished,+5.689,1:04.198,1:03.096,1:02.951,Mercedes,red_bull_ring,12,4,0
4,2020,1,Carlos Sainz,63,1:07.974,228.687,Finished,+8.903,1:04.537,1:03.971,1:03.971,McLaren,red_bull_ring,10,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016,20,Esteban Gutiérrez,None,None,None,Retired,Retired,1:21.401,NaN,NaN,Haas F1 Team,interlagos,0,21,0
1909,2016,20,Felipe Massa,None,None,None,Collision,Collision,1:20.423,1:20.151,1:20.032,Williams,interlagos,51,11,0
1910,2016,20,Kimi Räikkönen,None,None,None,Collision,Collision,1:19.554,1:19.936,1:19.376,Ferrari,interlagos,178,6,0
1911,2016,20,Marcus Ericsson,None,None,None,Collision,Collision,1:21.062,1:21.536,NaN,Sauber,interlagos,0,22,0


In [55]:
test

,Year,Round,Constructor,Constructor_Points,Constructor_Wins
0,2020,0,Mercedes,573,13
1,2020,0,Red Bull,319,2
2,2020,0,McLaren,202,0
3,2020,0,Racing Point,195,1
4,2020,0,Renault,181,0
...,...,...,...,...,...
1016,2016,20,Toro Rosso,63,0
1017,2016,20,Haas F1 Team,29,0
1018,2016,20,Renault,8,0
1019,2016,20,Sauber,2,0


I got the function working, now time to include the drivers best times (from qualifying Q1), will need to source that indivdually, but will add to these functions. After that I am done extracting the data I need. LET'S GO!! 

Also going to need to get tracks they raced on that round. 

Today's task is relitivly simple, going to get the remaining data, the qualifying laptimes and their 

I want to get the qualifying times for each racer and race as well, problem is that the order might just be messed up. I need to see how I would restructure the data so I can merge safely or see if I can merge two dataframes with a pandas function that finds the index automaticlly. 

I can use the pd.merge function to get what I need, just need to do a quick test to see if it would work. 

I want to merge the dataframes on the last names and the year and rounds. 


In [56]:
# These 3 cells grab all the information from the qualifying json of the Ergast Database. 
years = 2020
rounds = 9
done = "http://ergast.com/api/f1/{year}/{round}/qualifying.json"
done1 = done.format(year = str(years), round = rounds)
response = requests.get(done1)
test = json.loads(response.text)

In [57]:
test['MRData']['RaceTable']['Races'][0]['QualifyingResults'][0]['Q1']

'1:15.778'

In [58]:
test['MRData']['RaceTable']['Races'][0]['QualifyingResults'][0]['Driver']['familyName']
test['MRData']['RaceTable']['round']
test['MRData']['RaceTable']['season']
test['MRData']['RaceTable']['Races'][0]['QualifyingResults'][0]['Driver']['givenName']
test['MRData']['total']
test['MRData']['RaceTable']['Races'][0]['QualifyingResults'][0]['Constructor']

{'constructorId': 'mercedes',
 'url': 'http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One',
 'name': 'Mercedes',
 'nationality': 'German'}

In [59]:
# Now grabbing what I can from the constructor standings page from the API. 

url = "http://ergast.com/api/f1/{year}/{round}/constructorStandings.json".format(year = str(2019), round = str(5))
response = requests.get(url)
test1 = json.loads(response.text)

In [60]:
test1['MRData']['StandingsTable']

{'season': '2019',
 'round': '5',
 'StandingsLists': [{'season': '2019',
   'round': '5',
   'ConstructorStandings': [{'position': '1',
     'positionText': '1',
     'points': '217',
     'wins': '5',
     'Constructor': {'constructorId': 'mercedes',
      'url': 'http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One',
      'name': 'Mercedes',
      'nationality': 'German'}},
    {'position': '2',
     'positionText': '2',
     'points': '121',
     'wins': '0',
     'Constructor': {'constructorId': 'ferrari',
      'url': 'http://en.wikipedia.org/wiki/Scuderia_Ferrari',
      'name': 'Ferrari',
      'nationality': 'Italian'}},
    {'position': '3',
     'positionText': '3',
     'points': '87',
     'wins': '0',
     'Constructor': {'constructorId': 'red_bull',
      'url': 'http://en.wikipedia.org/wiki/Red_Bull_Racing',
      'name': 'Red Bull',
      'nationality': 'Austrian'}},
    {'position': '4',
     'positionText': '4',
     'points': '22',
     'wins': '0',
     'Const

## Explaining the GetConstructorStanding Function. 

This function takes from another part of the database, this would grab the information of how those constructors are doing in that specific round of a season (the input data).

From there, it would take in all the information from the json, convert into a dict (done automaticly on python since dicts are pretty much jsons) and then take the data that is important, and log the data inside a dataframe that will be exported to be later merged with the main dataframe. 

In [37]:
test1 = GetConstructorStanding(2019, 9)

In [38]:
test

{'MRData': {'xmlns': 'http://ergast.com/mrd/1.4',
  'series': 'f1',
  'url': 'http://ergast.com/api/f1/2020/9/qualifying.json',
  'limit': '30',
  'offset': '0',
  'total': '20',
  'RaceTable': {'season': '2020',
   'round': '9',
   'Races': [{'season': '2020',
     'round': '9',
     'url': 'https://en.wikipedia.org/wiki/2020_Tuscan_Grand_Prix',
     'raceName': 'Tuscan Grand Prix',
     'Circuit': {'circuitId': 'mugello',
      'url': 'http://en.wikipedia.org/wiki/Mugello_Circuit',
      'circuitName': 'Autodromo Internazionale del Mugello',
      'Location': {'lat': '43.9975',
       'long': '11.3719',
       'locality': 'Mugello',
       'country': 'Italy'}},
     'date': '2020-09-13',
     'time': '13:10:00Z',
     'QualifyingResults': [{'number': '44',
       'position': '1',
       'Driver': {'driverId': 'hamilton',
        'permanentNumber': '44',
        'code': 'HAM',
        'url': 'http://en.wikipedia.org/wiki/Lewis_Hamilton',
        'givenName': 'Lewis',
        'familyNa

## Since I got the standings of the constructor teams... 

Now I'm going to need to merge the two dataframes into one mega dataframe

## Updating the times 

At first I was going to drop the Q1 Qual time entirely and replace it with fastest time but I realized that it isn't any kind of data leak this data is only going help the model a bit more. So having the Qualifying time (Q1) and Fastest lap time in junction with each other. 

In [62]:
# This link is going to need a round and the year.... 
def GetFastestLap(round, year):
  url = "http://ergast.com/api/f1/{year}/{round}/fastest/1/results".format(year=year, round=round)

In [63]:
# Now I'm going to run a test, see if I can these two dataframes merge
get_qualifying(2017, 20)

,Year,Round,Driver,Q1_Time,Q2_Time,Q3_Time,Constructor
0,2017,21,Valtteri Bottas,1:37.356,1:36.822,1:36.231,Mercedes
1,2017,21,Lewis Hamilton,1:37.391,1:36.742,1:36.403,Mercedes
2,2017,21,Sebastian Vettel,1:37.817,1:37.023,1:36.777,Ferrari
3,2017,21,Daniel Ricciardo,1:38.016,1:37.583,1:36.959,Red Bull
4,2017,21,Kimi Räikkönen,1:37.453,1:37.302,1:36.985,Ferrari
5,2017,21,Max Verstappen,1:38.021,1:37.777,1:37.328,Red Bull
6,2017,21,Nico Hülkenberg,1:38.781,1:38.138,1:38.282,Renault
7,2017,21,Sergio Pérez,1:38.601,1:38.359,1:38.374,Force India
8,2017,21,Esteban Ocon,1:38.896,1:38.392,1:38.397,Force India
9,2017,21,Felipe Massa,1:38.629,1:38.565,1:38.550,Williams


In [65]:
# time to test out the merger
df1 = get_qualifying(2020, 4)
df2 = GetStandings(2020, 4)

In [68]:
df3 = pd.merge(df1, df2, on=['Year', 'Round', 'Driver'])

My Function works! time to put this function into the main function! Getting the data I need is now done 

In [71]:
# Now it's time to export my data and apply a base model 

df.to_csv("Data/driver-info.csv")

I got al the data I need now, time to fit a base model and see the results that I get out of it. I'm excited to see the base results

Things to note, I'm trying to predict what place each driver ends up in, and also their qualifying times, which are both regressional problems. I will need to build two models that work with that. 

In [45]:
!pip install category_encoders

In [72]:
# time to fit a base model
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.model_selection import train_test_split



# Setting my X variable and my y variable
X = df.drop('Position', axis=1)
y = df['Position']

# Using the One Hot Encoder 
encoder = ce.OrdinalEncoder()
X_encoded = encoder.fit_transform(X)


# starting out with a base x_train and y_train
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.25, random_state=42)


# now applying my model 
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [73]:
X_test

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Wins
1230,4,7,4,2.0,597.0,596.0,1,639,1111,830,7,1,18,210,4
111,1,6,14,-1.0,-1.0,-1.0,9,27,90,9,7,7,4,11,2
1846,5,17,8,-1.0,-1.0,-1.0,9,27,1682,9,7,14,22,11,2
1722,5,12,31,34.0,795.0,797.0,1,850,1564,1164,766,1,20,241,6
415,2,6,20,51.0,230.0,229.0,1,257,369,280,184,9,17,63,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1,11,20,11.0,114.0,113.0,1,131,178,134,88,9,9,62,1
802,3,5,18,-1.0,-1.0,-1.0,10,28,723,549,7,8,4,8,2
1626,5,7,17,-1.0,-1.0,-1.0,10,28,1471,1098,7,6,18,7,2
1828,5,17,4,49.0,846.0,849.0,1,901,1666,1241,813,1,22,230,7


In [74]:
# Let's test this 
y_pred = model.predict(X_test)

In [75]:
y_pred

array(['2', '18', '24', '1', '4', '4', '3', '7', '7', '3', '1', '6', '14',
       '16', '13', '20', '5', '20', '2', '14', '17', '3', '6', '8', '7',
       '5', '17', '16', '4', '13', '10', '7', '21', '14', '13', '20',
       '10', '1', '3', '10', '20', '16', '8', '5', '20', '22', '2', '3',
       '1', '17', '3', '20', '2', '2', '21', '20', '7', '12', '17', '22',
       '20', '14', '5', '12', '4', '14', '6', '11', '11', '8', '6', '4',
       '14', '12', '10', '1', '13', '20', '12', '10', '19', '16', '17',
       '20', '3', '22', '16', '12', '23', '16', '12', '4', '7', '6', '2',
       '11', '10', '1', '10', '18', '12', '19', '14', '11', '9', '2', '2',
       '15', '1', '20', '15', '7', '4', '15', '2', '13', '18', '16', '16',
       '14', '18', '14', '22', '16', '23', '2', '7', '2', '11', '9', '20',
       '7', '15', '14', '21', '10', '14', '12', '11', '14', '19', '12',
       '23', '12', '20', '19', '9', '14', '7', '16', '3', '5', '2', '10',
       '13', '4', '8', '9', '2', '5', '8', '2

In [76]:
X_pred = encoder.inverse_transform(X_test)

In [77]:
import numpy as np

y_pred_df = pd.DataFrame(y_pred, columns=["Position"])

In [78]:
X_pred['Position'] = y_pred

In [61]:
X_pred

,Year,Round,First_Name,Last_Name,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Wins,Position
1230,2017,7,Lewis,Hamilton,64,1:14.551,210.588,Finished,1:33:05.154,1:13.640,1:14.106,NaN,Mercedes,villeneuve,129,3,2
111,2020,6,Kimi,Räikkönen,NaN,NaN,NaN,+1 Lap,+1 Lap,1:28.493,NaN,NaN,Alfa Romeo,catalunya,0,0,18
1846,2016,17,Esteban,Ocon,NaN,NaN,NaN,+1 Lap,+1 Lap,1:36.451,NaN,NaN,Manor Marussia,suzuka,0,0,24
1722,2016,12,Nico,Rosberg,51,1:19.122,208.114,Finished,+15.845,1:33.302,1:22.806,1:19.965,Mercedes,hockenheimring,198,5,1
415,2019,6,Max,Verstappen,9,1:16.229,157.593,Finished,+5.537,1:17.244,1:16.726,1:16.357,Red Bull,monaco,78,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2020,11,Max,Verstappen,60,1:28.139,210.267,Finished,+4.470,1:33.630,1:33.157,1:31.867,Red Bull,nurburgring,147,1,3
802,2018,5,Lance,Stroll,NaN,NaN,NaN,+2 Laps,+2 Laps,1:44.359,1:43.585,NaN,Williams,catalunya,4,0,18
1626,2016,7,Kevin,Magnussen,NaN,NaN,NaN,+2 Laps,+2 Laps,1:16.253,1:16.058,NaN,Renault,villeneuve,6,0,15
1828,2016,17,Lewis,Hamilton,36,1:35.152,219.703,Finished,+5.776,1:34.444,1:33.046,1:32.850,Mercedes,suzuka,280,6,2


# Where I left off 

Looking at where I left off, it's clear that I got a base model running. This is a regressional problem, and my base model is not very accurate. I also remembered that since the model spits out a numpy array, I can attach it to my test dataframe so it can predict the places for what place each driver will get. 

Now I'm going to make another data frame, this one will be the values it's going to take to actually predict. then let's see what the model will do.

In [81]:
# let's see how good hamilton does in red bull ring
mask = df['Track'] == 'red_bull_ring'  
mask2 = df['Driver'] == 'Lewis Hamilton'
df2 = df[mask]
df2[mask2]

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Position,Wins
3,2020,1,Lewis Hamilton,67,1:07.712,229.572,Finished,+5.689,1:04.198,1:03.096,1:02.951,Mercedes,red_bull_ring,12,4,0
20,2020,2,Lewis Hamilton,68,1:06.719,232.989,Finished,1:22:50.683,1:04.198,1:03.096,1:02.951,Mercedes,red_bull_ring,37,2,1
476,2019,9,Lewis Hamilton,69,1:08.050,228.432,Finished,+22.805,1:30.609,1:29.520,1:28.319,Mercedes,red_bull_ring,197,1,6
887,2018,9,Lewis Hamilton,None,None,None,Fuel pressure,Fuel pressure,1:31.271,1:30.645,1:30.029,Mercedes,red_bull_ring,145,2,3
1272,2017,9,Lewis Hamilton,69,1:07.411,230.597,Finished,+7.430,1:41.983,1:41.275,1:40.593,Mercedes,red_bull_ring,151,2,3
1654,2016,9,Lewis Hamilton,67,1:08.411,227.647,Finished,1:27:38.107,1:44.259,1:43.526,2:01.954,Mercedes,red_bull_ring,142,2,3


In [86]:
column_list = ["Year", "Round", "Driver", "Track", "Points", "Wins", "Q1_Time", "Q2_Time", "Q3_Time"]
test_df = pd.DataFrame(columns=column_list)

new_info = {"Year": 2021, 
            "Round": 3,
            "Driver": "Lewis Hamilton",
            "Constructor": "Mercedes",
            "FastestTime": "1:07.342",
            "FastestLap": "66",
            "FastestLapAvgSpeedKPH": "222",
            'status': "Finished",
            'TimeBehind': "1:20:05.154",
            "Q1_Time": "1:04.432",
            "Q2_Time": "1:04.432",
            "Q3_Time": "1:04.432",
            "Track": "imola",
            "Points": 0,
            "Wins": 0}

test_df = test_df.append(new_info, ignore_index=True)
# Always make sure to encode my dataframe, 
# I can decode it after I'm done. 
test_df_e = encoder.fit_transform(test_df)

In [87]:
# getting the prediction, let's see how well he does
y_pred = model.predict(test_df_e)
y_pred

array(['1'], dtype=object)

# Play with the dict and see how it effects the model

The only features that are important to this model is the name and which round it is. I thought the wins and points would be the most important parts to the model but it's clear that it's doing it based on the driver. 

I want to try a tree model that will be much more accurate. and from there see what I can do to get the most accurate model possible. 



# TODO

Use the Data to create the second, base classifcation model 


In [88]:
# Notes: This is a multiclassifcation problem, We are trying to predict 3 different outcomes, either Top 3, In the points,
# or out of the points. 

# First of since that is the outcome we are going for we need to change the data accordingly. 
# Creating a new column called "Results". To do that I will make a function that will add all of the results
def create_results(df):
    # Looping throughout the dataframe to check the results of each driver. 
    df['Position'] = pd.to_numeric(df['Position'])

    if df['Position'] <= 3:
        df['Result'] = "Podium"
    if df['Position'] > 3 & df['Position'] <= 10:
        df['Result'] = 'In the Points'
    if df['Position'] > 10:
        df['Result'] = "Out of the Points"
    return df

In [89]:
# This .apply function takes the lambda loop and creates the result column. 
df = df.apply(lambda df: create_results(df), axis=1)

In [90]:
# Dropping the Position column since we don't need that anymore. 
df_class = df.drop(columns='Position')

In [91]:
# Testing to see how it comes out. 
df_class

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Wins,Result
0,2020,1,Valtteri Bottas,68,1:07.657,229.758,Finished,1:30:55.739,1:04.111,1:03.015,1:02.939,Mercedes,red_bull_ring,25,1,Podium
1,2020,1,Charles Leclerc,64,1:07.901,228.933,Finished,+2.700,1:04.500,1:04.041,1:03.923,Ferrari,red_bull_ring,18,0,Podium
2,2020,1,Lando Norris,71,1:07.475,230.378,Finished,+5.491,1:04.606,1:03.819,1:03.626,McLaren,red_bull_ring,16,0,Podium
3,2020,1,Lewis Hamilton,67,1:07.712,229.572,Finished,+5.689,1:04.198,1:03.096,1:02.951,Mercedes,red_bull_ring,12,0,In the Points
4,2020,1,Carlos Sainz,63,1:07.974,228.687,Finished,+8.903,1:04.537,1:03.971,1:03.971,McLaren,red_bull_ring,10,0,In the Points
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016,20,Esteban Gutiérrez,None,None,None,Retired,Retired,1:21.401,NaN,NaN,Haas F1 Team,interlagos,0,0,Out of the Points
1909,2016,20,Felipe Massa,None,None,None,Collision,Collision,1:20.423,1:20.151,1:20.032,Williams,interlagos,51,0,Out of the Points
1910,2016,20,Kimi Räikkönen,None,None,None,Collision,Collision,1:19.554,1:19.936,1:19.376,Ferrari,interlagos,178,0,In the Points
1911,2016,20,Marcus Ericsson,None,None,None,Collision,Collision,1:21.062,1:21.536,NaN,Sauber,interlagos,0,0,Out of the Points


In [93]:
def quick_encode(y):
    if y == "Podium":
        y = 1
    if y == 'In the Points':
        y = 2
    if y == "Out of the Points":
        y = 3 
    return y 

def quick_decode(y):
    if y == 1:
        y = "Podium"
    if y == 2:
        y = 'In the Points'
    if y == 3:
        y = 'Out of the Points'
    return y

In [94]:
# Importing the Random Forest model
from sklearn.ensemble import RandomForestClassifier

# Now building our second classifcation model. 
X = df_class.drop('Result', axis=1)
y = df_class['Result']

# Using the One Hot Encoder 
encoder = ce.OrdinalEncoder()
X_encoded = encoder.fit_transform(X)
y_encoded = y.apply(lambda y: quick_encode(y))

# starting out with a base x_train and y_train
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.25, random_state=42)


# now applying my model 
model2 = RandomForestClassifier()
model2.fit(X_train, y_train)

RandomForestClassifier()

In [95]:
# Let's test this 
y_pred = model2.predict(X_test)
y_pred

array([1, 3, 3, 1, 2, 2, 1, 2, 2, 1, 1, 2, 3, 3, 3, 3, 2, 3, 1, 3, 3, 1,
       2, 2, 2, 2, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3, 2, 1, 1, 3, 3, 3, 3, 2,
       3, 3, 1, 1, 1, 3, 1, 3, 1, 1, 3, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3,
       2, 3, 2, 2, 2, 2, 3, 3, 2, 1, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3,
       3, 3, 2, 2, 2, 2, 1, 3, 3, 1, 2, 3, 3, 3, 3, 2, 3, 1, 1, 3, 1, 3,
       3, 2, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 1, 3, 2, 3, 2,
       3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 1, 2, 1, 3,
       2, 2, 2, 2, 1, 2, 2, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 1, 2, 2, 1, 3,
       3, 3, 3, 3, 1, 2, 2, 3, 3, 1, 2, 3, 3, 2, 3, 2, 3, 2, 2, 3, 3, 3,
       2, 3, 2, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 3,
       3, 3, 3, 2, 2, 3, 2, 3, 3, 3, 2, 3, 2, 1, 3, 3, 3, 3, 3, 2, 2, 2,
       2, 1, 3, 2, 3, 3, 2, 2, 1, 2, 1, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2,
       2, 3, 3, 3, 3, 1, 2, 1, 2, 3, 3, 3, 2, 3, 3, 2, 1, 3, 2, 1, 2, 3,
       3, 2, 2, 1, 3, 2, 3, 3, 3, 2, 2, 2, 3, 3, 3,

In [96]:
# Testing to see how position compares
test_df = encoder.inverse_transform(X_test)
y_df = pd.DataFrame(y_pred)
y_df = y_df[0].apply(lambda y_df: quick_decode(y_df))
y_df
X_pred = X_test
X_pred['Result'] = y_df
X_pred

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Wins,Result
1230,2017,7,4,2.0,597.0,596.0,1,639,1111,830,7,1,18,210,4,NaN
111,2020,6,14,-1.0,-1.0,-1.0,9,27,90,9,7,7,4,11,2,In the Points
1846,2016,17,8,-1.0,-1.0,-1.0,9,27,1682,9,7,14,22,11,2,NaN
1722,2016,12,31,34.0,795.0,797.0,1,850,1564,1164,766,1,20,241,6,NaN
415,2019,6,20,51.0,230.0,229.0,1,257,369,280,184,9,17,63,2,Out of the Points
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2020,11,20,11.0,114.0,113.0,1,131,178,134,88,9,9,62,1,Out of the Points
802,2018,5,18,-1.0,-1.0,-1.0,10,28,723,549,7,8,4,8,2,NaN
1626,2016,7,17,-1.0,-1.0,-1.0,10,28,1471,1098,7,6,18,7,2,NaN
1828,2016,17,4,49.0,846.0,849.0,1,901,1666,1241,813,1,22,230,7,NaN


## I got both models working. Now I need to start working on deploying them. 

In [97]:
df.to_csv("driver-info.csv")

In [98]:
df

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Position,Wins,Result
0,2020,1,Valtteri Bottas,68,1:07.657,229.758,Finished,1:30:55.739,1:04.111,1:03.015,1:02.939,Mercedes,red_bull_ring,25,1,1,Podium
1,2020,1,Charles Leclerc,64,1:07.901,228.933,Finished,+2.700,1:04.500,1:04.041,1:03.923,Ferrari,red_bull_ring,18,2,0,Podium
2,2020,1,Lando Norris,71,1:07.475,230.378,Finished,+5.491,1:04.606,1:03.819,1:03.626,McLaren,red_bull_ring,16,3,0,Podium
3,2020,1,Lewis Hamilton,67,1:07.712,229.572,Finished,+5.689,1:04.198,1:03.096,1:02.951,Mercedes,red_bull_ring,12,4,0,In the Points
4,2020,1,Carlos Sainz,63,1:07.974,228.687,Finished,+8.903,1:04.537,1:03.971,1:03.971,McLaren,red_bull_ring,10,5,0,In the Points
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016,20,Esteban Gutiérrez,None,None,None,Retired,Retired,1:21.401,NaN,NaN,Haas F1 Team,interlagos,0,21,0,Out of the Points
1909,2016,20,Felipe Massa,None,None,None,Collision,Collision,1:20.423,1:20.151,1:20.032,Williams,interlagos,51,11,0,Out of the Points
1910,2016,20,Kimi Räikkönen,None,None,None,Collision,Collision,1:19.554,1:19.936,1:19.376,Ferrari,interlagos,178,6,0,In the Points
1911,2016,20,Marcus Ericsson,None,None,None,Collision,Collision,1:21.062,1:21.536,NaN,Sauber,interlagos,0,22,0,Out of the Points


In [100]:
# Pickling the model using joblib
import joblib

filename1 = "models/modelPosition.sav"
filename2 = "models/modelPoints.sav"

# Dumping both models 
joblib.dump(model, filename1)
joblib.dump(model2, filename2)

['models/modelPoints.sav']

## Running into an issue with my predict function, this time around since the user input is a class of a dicttionary. It is having a hard time becoming a dataframe. This next few cells will be looking at how I'm going to convert this class into dictionary to convert into a dataframe. 

* Turns out Pydantic has a .dict() fucntion built into the class, trying to figure out how to correctly use it. 

## New bug found, when trying to run the second function, something goes wrong and the code doesn't think the dict is a dict. odd but I might have to change the user input possibly? 

In [108]:
# loading in the model
bugfix = joblib.load("models/modelPoints.sav")

In [109]:
# Seems like no error pops up this time around, time to see how it would work with the test data. 
new_info = {"Year": 2021, 
            "Round": 3,
            "Driver": "Max Verstappen",
            "Constructor": "Red Bull",
            "FastestTime": "1:07.342",
            "FastestLap": "66",
            "FastestLapAvgSpeedKPH": "222",
            'status': "Finished",
            'TimeBehind': "1:20:05.154",
            "Q1_Time": "1:04.432",
            "Q2_Time": "1:04.432",
            "Q3_Time": "1:04.432",
            "Track": "imola",
            "Points": 0,
            "Wins": 0}

test_df = test_df.append(new_info, ignore_index=True)
# Always make sure to encode my dataframe, 
# I can decode it after I'm done. 
test_df_e = encoder.fit_transform(test_df)

In [110]:
bugfix.predict(test_df_e)

ValueError: could not convert string to float: 'Max'

In [111]:
# Looks like it manages to predict it just fine, must be something with the API. Going to have to make a new router possibly! 
test_df

,Year,Round,Driver,FastestLap,FastestTime,FastestLapAvgSpeedKPH,status,TimeBehind,Q1_Time,Q2_Time,Q3_Time,Constructor,Track,Points,Wins,First_Name,Last_Name
0,2017,7,Lewis Hamilton,64,1:14.551,210.588,Finished,1:33:05.154,1:13.640,1:14.106,NaN,Mercedes,villeneuve,129,3,NaN,NaN
1,2020,6,Kimi Räikkönen,NaN,NaN,NaN,+1 Lap,+1 Lap,1:28.493,NaN,NaN,Alfa Romeo,catalunya,0,0,NaN,NaN
2,2016,17,Esteban Ocon,NaN,NaN,NaN,+1 Lap,+1 Lap,1:36.451,NaN,NaN,Manor Marussia,suzuka,0,0,NaN,NaN
3,2016,12,Nico Rosberg,51,1:19.122,208.114,Finished,+15.845,1:33.302,1:22.806,1:19.965,Mercedes,hockenheimring,198,5,NaN,NaN
4,2019,6,Max Verstappen,9,1:16.229,157.593,Finished,+5.537,1:17.244,1:16.726,1:16.357,Red Bull,monaco,78,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2016,12,Daniel Ricciardo,48,1:18.442,209.918,Finished,+6.996,1:39.968,1:23.234,1:20.280,Red Bull,hockenheimring,133,0,NaN,NaN
479,2021,3,NaN,66,1:07.342,222,Finished,1:20:05.154,1:04.432,1:04.432,1:04.432,Red Bull,imola,0,0,Max,Verstappen
480,2021,3,Max Verstappen,66,1:07.342,222,Finished,1:20:05.154,1:04.432,1:04.432,1:04.432,Red Bull,imola,0,0,NaN,NaN
481,2021,3,Max Verstappen,66,1:07.342,222,Finished,1:20:05.154,1:04.432,1:04.432,1:04.432,Red Bull,imola,0,0,NaN,NaN
